In [1]:
# Ir um por vez

!gdown '1TIOo9IXEa1siX-VZi9tXnupsDs64OJlp'
!unzip "/content/Unico.zip" -d "/content/Unico"
!rm -rf '/content/Unico.zip'

Downloading...
From: https://drive.google.com/uc?id=1TIOo9IXEa1siX-VZi9tXnupsDs64OJlp
To: /content/Unico.zip
100% 1.26G/1.26G [00:09<00:00, 127MB/s]
Archive:  /content/Unico.zip
  inflating: /content/Unico/Unico.npy  


In [2]:
import numpy as np
import scipy as sp
from scipy import signal
from scipy.signal import hilbert
import matplotlib.pyplot as plt

In [3]:
def segmenta(sinal, passo, dimensoes):
  #Inputs
  #- sinal: sinal a ser segmentado
  #- passo: quantas amostras irá avançar de um segmento para outro
  #- dimensoes: Dimensões esperada do output

  #Output
  #- segmentado: Sinal segmentado em segmentos de 2*passo de duração e com
  # overlap de 1 passo entre eles

  segmentado = np.zeros(dimensoes)
  cont = 0
  for linha in range(sinal.shape[0]):
      sinal_s = sinal[linha,:,:]
      for i in range(1000000):
          if ~np.any(sinal_s[i*passo:(i+2)*passo,:]) or sinal_s[i*passo:(i+2)*passo,:].shape[0] != int(2*passo):
              break
          segmentado[cont,:,:] = sinal_s[i*passo:(i+2)*passo,:]
          cont = cont+1
  return segmentado

def Features(sinal,fs=500):
  #Inputs
  #- sinal: sinal de onde serão tiradas as características
  #- fs: frequência de amostragem do sinal

  #Output
  #- sig_features: 

  sig_features = np.zeros((256,20))

  # A extração de características é feita canal por canal
  for i in range(20):
    sinal_c = sinal[:,i]

    # Encontra a frequência instantanea do canal utilizando a transformada de Hilbert
    analytic_signal = hilbert(sinal_c)
    analytic_signal = np.abs(analytic_signal)
    instantaneous_phase = np.unwrap(np.angle(analytic_signal))
    instantaneous_frequency = np.abs((np.diff(instantaneous_phase) / (2.0*np.pi) * fs))
    instantaneous_frequency = instantaneous_frequency.reshape((-1,1))
    

    # Determina a densidade do espectro de potência do canal
    f, psd = signal.welch(sinal_c, 500)

    # f, psd = sp.signal.periodogram(sinal_c,freq=250)

    # alfa = (f>=8) & (f<12)
    # alfa = psd[alfa].sum().reshape((-1,1))

    # beta = (f>=12) & (f<30)
    # beta = psd[beta].sum().reshape((-1,1))
    
    # gamma = (f>=30)
    # gamma = psd[gamma].sum().reshape((-1,1))

    # delta = (f>=2) & (f<4)
    # delta = psd[delta].sum().reshape((-1,1))

    # theta = (f>=4) & (f<8)
    # theta = psd[theta].sum().reshape((-1,1))

    alfa = np.trapz(y = psd[4:6], x = f[4:6]).reshape((-1,1))
    beta = np.trapz(y = psd[6:15], x = f[6:15]).reshape((-1,1))
    gamma = np.trapz(y = psd[15:50], x = f[15:50]).reshape((-1,1))
    delta = np.trapz(y = psd[0:2], x = f[0:2]).reshape((-1,1))
    theta = np.trapz(y = psd[2:4], x = f[2:4]).reshape((-1,1))

    sig_mean = np.mean(sinal_c).reshape((-1,1))

    sig_std = np.std(sinal_c).reshape((-1,1))
    


    # # Determina a entropia dispersa do sinal (dim = 1, 1) 
    # dist, phi = EH.DispEn(sinal_c)
    # dist = dist.reshape((-1,1))
    # phi = phi.reshape((-1,1))

    # # Determina a entropia aproximada do sinal (dim = 3,4)
    # app, phi_app = EH.ApEn(sinal_c)
    # app = app.reshape((-1,1))
    # phi_app = phi_app.reshape((-1,1))

    # feat = np.concatenate((instantaneous_frequency,psd,sig_mean,sig_std,dist,phi,app,phi_app)).reshape((-1,1))
    feat = np.concatenate((instantaneous_frequency,delta,theta,alfa,beta,gamma,sig_mean,sig_std)).reshape((-1,1))

    

    sig_features[:,i] = feat[:,0]
  
  return sig_features

In [4]:
geral = np.load('/content/Unico/Unico.npy')

np.random.seed(42)

np.random.shuffle(geral)

In [5]:
geral_seg = segmenta(geral,125,(70000,250,22))
lim = np.where(geral_seg[:,0,0]==0)[0][0]
geral_seg = geral_seg[:lim,:,:]

In [6]:
x_data, y_data = geral_seg[:,:,:20], geral_seg[:,:,-1]
y_data = np.mean(y_data, axis=1,dtype=np.int16).reshape((-1,1))

In [12]:
x_data.shape

(61699, 250, 20)

In [13]:
data_signals = np.zeros((x_data.shape[0],256,20))
for segm in range(x_data.shape[0]):
    data_signals[segm,:,:] = Features(x_data[segm,:,:])

In [14]:
np.save('Signals',data_signals)
np.save('Labels',y_data)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

!cp '/content/Signals.npy' '/content/drive/MyDrive/EEG - IC TCC/Dados'
!cp '/content/Labels.npy' '/content/drive/MyDrive'

Mounted at /content/drive
